In [1]:
import json
import logging
import os
import sys
from datetime import datetime, timezone


from graphiti_core import Graphiti
from graphiti_core.llm_client.groq_client import GroqClient, LLMConfig
from graphiti_core.embedder.openai import OpenAIEmbedder, OpenAIEmbedderConfig
from graphiti_core.cross_encoder.openai_reranker_client import OpenAIRerankerClient

from graphiti_core.edges import EntityEdge
from graphiti_core.nodes import EpisodeType

In [2]:
def setup_logging():
    logger = logging.getLogger()
    logger.setLevel(logging.ERROR)
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    return logger

llm_config=LLMConfig(
    api_key = os.getenv('GROQ_API_KEY', 'value does not exist'),
    model="llama-3.3-70b-versatile",
    small_model="llama-3.1-8b-instant",
    temperature = 0.1)

llm_client = GroqClient(config=llm_config)

graphiti = Graphiti(
    "bolt://localhost:7687",
    "neo4j",
    "123456789", 
    llm_client=llm_client,
    embedder=OpenAIEmbedder(
        config=OpenAIEmbedderConfig(
            api_key="abc",
            embedding_model="mxbai-embed-large",
            embedding_dim=512,
            base_url="http://localhost:11434/v1",
        )
    ),
    cross_encoder=OpenAIRerankerClient(client=llm_client, config=llm_config),
)

In [3]:
try:
    await graphiti.build_indices_and_constraints()
    print("Graphiti indices built successfully.")
except Exception as e:
    print(f"Note: {str(e)}")
    print("Continuing with existing indices...")

Graphiti indices built successfully.


In [ ]:
episodes = [
    {
        'content': '"Carlos Alcántara" is an actor, he was born in 1964, he acted in "Pataclaun" as the character "Machín".',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
]

In [ ]:
episodes = [
    {   
        'name': 'Real information',
        'content': '"Armando Machuca" is an actor, he was born in 1976, '
        'he was who acted in "Pataclaun" as the character "Machín" '
        '(november 30th, 1997  - december 19th, 1999)',
        'type': EpisodeType.text,
        'description': 'podcast transcript',
    },
]

In [ ]:
async def ingest_data(client: Graphiti):

    for i, episode in enumerate(episodes):
        await graphiti.add_episode(
            name=episode['name'],
            episode_body=episode['content']
            if isinstance(episode['content'], str)
            else json.dumps(episode['content']),
            source=episode['type'],
            source_description=episode['description'],
            reference_time=datetime.now(timezone.utc),
        )
        print(f'Added episode: {episode["name"], str} ({episode["type"].value})')
        

await ingest_data(graphiti)

In [4]:
await graphiti.close()